In [121]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [122]:
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [123]:
df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [125]:
df.describe()

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,20758.00000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000
mean,10378.50000,23.841804,1.700245,87.887768,2.445908,2.761332,2.029418,0.981747,0.616756
std,5992.46278,5.688072,0.087312,26.379443,0.533218,0.705375,0.608467,0.838302,0.602113
min,0.00000,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,5189.25000,20.000000,1.631856,66.000000,2.000000,3.000000,1.792022,0.008013,0.000000
50%,10378.50000,22.815416,1.700000,84.064875,2.393837,3.000000,2.000000,1.000000,0.573887
75%,15567.75000,26.000000,1.762887,111.600553,3.000000,3.000000,2.549617,1.587406,1.000000
max,20757.00000,61.000000,1.975663,165.057269,3.000000,4.000000,3.000000,3.000000,2.000000


In [126]:
df.drop('id',axis=1,inplace=True)
# df_test.drop('id',axis=1,inplace=True)

>DataFrame.select_dtypes(include=None, exclude=None) \
>Return a subset of the DataFrame’s columns based on the column dtypes.

In [127]:
target='NObeyesdad'
obj_col=[x for x in df.select_dtypes(include=['object']) if x !=target]
num_col=[x for x in df.select_dtypes(exclude=['object']) if x !=target]

In [128]:
print(df['NObeyesdad'].unique())
for i in obj_col:
    print(i,":",df[i].unique())

['Overweight_Level_II' 'Normal_Weight' 'Insufficient_Weight'
 'Obesity_Type_III' 'Obesity_Type_II' 'Overweight_Level_I'
 'Obesity_Type_I']
Gender : ['Male' 'Female']
family_history_with_overweight : ['yes' 'no']
FAVC : ['yes' 'no']
CAEC : ['Sometimes' 'Frequently' 'no' 'Always']
SMOKE : ['no' 'yes']
SCC : ['no' 'yes']
CALC : ['Sometimes' 'no' 'Frequently']
MTRANS : ['Public_Transportation' 'Automobile' 'Walking' 'Motorbike' 'Bike']


In [129]:
feature_dict={
    'Gender':{
        'Female':0,
        'Male':1
    },
    'family_history_with_overweight':{
        'yes':1,
        'no':0
    },
    'FAVC':{
        'yes':1,
        'no':0
    },
    'CAEC':{
        'no':0,
        'Sometimes':1,
        'Frequently':2,
        'Always':3
    },
    'SMOKE':{
        'no':0, 
        'yes':1
    },
    'SCC':{
        'no':1,
        'yes':0
    },
    'CALC':{
       'no':0,
       'Sometimes':1,
       'Always':2 
    },
    'MTRANS' : {
        'Public_Transportation':2,
        'Automobile':4,
        'Walking':0,
        'Motorbike':3,
        'Bike':1
        },
    'NObeyesdad':{
    'Insufficient_Weight': 0,
    'Normal_Weight': 1,
    'Overweight_Level_I': 2,
    'Overweight_Level_II': 3,
    'Obesity_Type_I': 4,
    'Obesity_Type_II': 5,
    'Obesity_Type_III': 6,
}
}

obesity_dict = {
    'Insufficient_Weight': 0,
    'Normal_Weight': 1,
    'Overweight_Level_I': 2,
    'Overweight_Level_II': 3,
    'Obesity_Type_I': 4,
    'Obesity_Type_II': 5,
    'Obesity_Type_III': 6,
}

inverse_obesity_dict = {
    0: 'Insufficient_Weight',
    1: 'Normal_Weight',
    2: 'Overweight_Level_I',
    3: 'Overweight_Level_II',
    4: 'Obesity_Type_I',
    5: 'Obesity_Type_II',
    6: 'Obesity_Type_III' 
}

for feature, encoding_map in feature_dict.items():
    print(feature,encoding_map)
    df[feature] = df[feature].map(encoding_map)

Gender {'Female': 0, 'Male': 1}
family_history_with_overweight {'yes': 1, 'no': 0}
FAVC {'yes': 1, 'no': 0}
CAEC {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
SMOKE {'no': 0, 'yes': 1}
SCC {'no': 1, 'yes': 0}
CALC {'no': 0, 'Sometimes': 1, 'Always': 2}
MTRANS {'Public_Transportation': 2, 'Automobile': 4, 'Walking': 0, 'Motorbike': 3, 'Bike': 1}
NObeyesdad {'Insufficient_Weight': 0, 'Normal_Weight': 1, 'Overweight_Level_I': 2, 'Overweight_Level_II': 3, 'Obesity_Type_I': 4, 'Obesity_Type_II': 5, 'Obesity_Type_III': 6}


In [130]:
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,1,0,2.763573,1,0.000000,0.976473,1.0,2,3
1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,2,0,2.000000,1,1.000000,1.000000,0.0,4,1
2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,1,0,1.910378,1,0.866045,1.673584,0.0,2,0
3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,1,0,1.674061,1,1.467863,0.780199,1.0,2,6
4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,1,0,1.979848,1,1.967973,0.931721,1.0,2,3


In [131]:
X = df.drop(['NObeyesdad'], axis=1)
y = df['NObeyesdad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [132]:
# Define XGBClassifier
xgb_0 = XGBClassifier(
    learning_rate=0.1,
    n_estimators=400,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    colsample_bytree=0.8,
    subsample=0.8,
    objective='multi:softprob',
    nthread=-1,
    random_state=42
)

# Train the classifier
xgb_0.fit(X_train, y_train)

# Make predictions on the test set
predictions = xgb_0.predict(X_test)

# Convert the scikit-learn model to a booster
bst = xgb_0.get_booster()

# Evaluate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9161849710982659


In [133]:
df_test.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [134]:

for feature, encoding_map in feature_dict.items(): 
    if feature !='NObeyesdad':
        df_test[feature] = df_test[feature].map(encoding_map)

In [142]:
df_test.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,1,0,2.825629,1,0.855400,0.000000,1.0,2
1,20759,0,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,1,0,3.000000,1,1.000000,0.000000,1.0,2
2,20760,0,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,1,0,2.621877,1,0.000000,0.250502,1.0,2
3,20761,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,1,0,2.786417,1,0.094851,0.000000,1.0,2
4,20762,0,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,1,0,2.653531,1,0.000000,0.741069,1.0,2


In [143]:
prediction_test=xgb_0.predict(df_test.drop('id',axis=1))

In [144]:
print(len(prediction_test))

13840


In [145]:
df_test['NObeyesdad']=prediction_test

In [146]:
df_test.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,20758,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,1,0,2.825629,1,0.855400,0.000000,1.0,2,5
1,20759,0,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,1,0,3.000000,1,1.000000,0.000000,1.0,2,2
2,20760,0,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,1,0,2.621877,1,0.000000,0.250502,1.0,2,6
3,20761,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,1,0,2.786417,1,0.094851,0.000000,1.0,2,4
4,20762,0,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,1,0,2.653531,1,0.000000,0.741069,1.0,2,6


In [147]:
df_test['NObeyesdad']=df_test['NObeyesdad'].map(inverse_obesity_dict)

In [148]:
df_test.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,20758,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,1,0,2.825629,1,0.855400,0.000000,1.0,2,Obesity_Type_II
1,20759,0,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,1,0,3.000000,1,1.000000,0.000000,1.0,2,Overweight_Level_I
2,20760,0,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,1,0,2.621877,1,0.000000,0.250502,1.0,2,Obesity_Type_III
3,20761,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,1,0,2.786417,1,0.094851,0.000000,1.0,2,Obesity_Type_I
4,20762,0,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,1,0,2.653531,1,0.000000,0.741069,1.0,2,Obesity_Type_III


In [152]:
submission=df_test[['id','NObeyesdad']]

In [154]:
submission.to_csv('submission.m1.csv', index = False)